In [2]:
%run GraphFamilies.ipynb

In [3]:
# Get list of graphs from graph families

#SETTINGS:
# Higher settings will increase accuracy but result in slower runtime

# Maximum size hole to include
maxhole = 10

# Maximum length path to include
maxpath = 5

# TRUE: Use connected graph families
# FALSE: Use disconnected graph familise
connected = True 

#############
graph_family_list = AllFamilies(maxhole, maxpath, connected)
forbidden_graphs = []
print(f"{len(graph_family_list)} graphs included for testing")

1722 graphs included for testing


In [4]:
# Helper functions

def get_all_subgraphs(G):
    G_sg_list = []

    for vertex_subset in Subsets(G.vertices()):
            current_sg = G.subgraph(vertex_subset)

            add_flag = True
            for H in G_sg_list:
                if (current_sg.order() != H.order()) or (current_sg.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                    continue
                elif H.is_isomorphic(current_sg):
                    add_flag = False
                    break 
            
            if add_flag == True:
                G_sg_list.append(current_sg)
    
    return G_sg_list

def get_best_sgs(graph_list = graph_family_list):
    # Need only to test graphs know contained in graph with the least subgraphs
    # Estimate using number of vertices
    # Find this graph
    best_order = 1000000
    for i in graph_list:
        current_order = i.order()
        if current_order < best_order:
            best_order = current_order
            best_graph = i
    
    return best_graph

def remove_redundant(tuple_list):
    cleaned_list = deepcopy(tuple_list)

    for i in tuple_list:
        for j in tuple_list:
            if tuple_list.index(i) == tuple_list.index(j): continue

            i_checklist = 0
            for x in i:
                for y in j:
                    if x.subgraph_search(y, induced=True) is not None:
                        i_checklist = i_checklist + 1
                        break
            
            if i_checklist == len(i):
                if j in cleaned_list: cleaned_list.remove(j)

    
    return cleaned_list

def check_exclusions(check_tuple, exclusion_list):

    for i in exclusion_list:
        i_checklist = 0

        for x in i:
            for y in check_tuple:
                if x.subgraph_search(y, induced=True) is not None:
                    i_checklist = i_checklist + 1
                    break
        
        if i_checklist == len(i):
            return True
    
    return False



In [8]:
# Find classification for single induced subgraph

def find_forbidden_1(graph_list = graph_family_list,exclusions=[]):
    forbidden_graphs = []
    test_sg_list = get_all_subgraphs(get_best_sgs(graph_list))
    
    # Order list of subgraphs we need to test by most vertices first
    test_sg_list.sort(reverse = True, key=order)

    # Check each 
    for test_sg in test_sg_list:
        add_flag = True

        # Make sure graph isn't excluded
        for exclusion in exclusions:
            if exclusion.subgraph_search(test_sg, induced=True) is not None:
                add_flag = False
                break
        if add_flag == False: continue
        
        # Make sure subgraph isn't already contained in a graph already included (most vertices first means this test is simple)
        for already_seen in forbidden_graphs:
            if already_seen.subgraph_search(test_sg, induced=True) is not None:
                add_flag = False
                break
        if add_flag == False: continue
        
        # Look at each graph in each family
        for current_graph in graph_list:
            if current_graph.subgraph_search(test_sg, induced=True) is None:
                add_flag = False
                break
            
        
        # Include subgraph only if found as a subgraph in all graphs
        if add_flag == True:
            forbidden_graphs.append(test_sg)
    
    return(forbidden_graphs)


forbidden_graphs.insert(1,find_forbidden_1())

In [20]:
def find_forbidden_k(k,graph_list = graph_family_list,exclusions=[]):
    forbidden_graph_k = []

    test_first_sg_list = get_all_subgraphs(get_best_sgs(graph_list))
    
    # Order list of subgraphs we need to test by most vertices first
    test_first_sg_list.sort(reverse = True, key=order)

    # Check each 
    for test_sg in test_first_sg_list:
        add_flag = True

        # Check that isn't in the monogenic case
        for case_1 in forbidden_graphs[1]:
            if case_1.subgraph_search(test_sg, induced=True) is not None:
                add_flag = False
                break
        if add_flag == False: continue

        # Create a new list by removing all graphs containing our subgraph
        culled_graph_list = deepcopy(graph_list)
        for current_graph in graph_list:
            good_flag = False

            # Check if the subgraph we're testing appears as a subgraph in this graph
            if current_graph.subgraph_search(test_sg, induced=True) is not None:
                good_flag = True
            
            # If it does, remove it from our new list
            if good_flag == True:
                culled_graph_list.remove(current_graph)
        
        # Find graph (k-1)-tuples which are in all graphs of this new table but not in the monogenic case
        if culled_graph_list == []: continue
        if k - 1 == 1:
            second_graphs = find_forbidden_1(culled_graph_list,forbidden_graphs[k-1])
        else:
            second_graphs = find_forbidden_k(k-1,culled_graph_list,forbidden_graphs[k-1])

        # Add ts
        for second_graph in second_graphs:
            t = [test_sg]
            if k == 2:
                t.append(second_graph)
            else:
                for H in second_graph:
                    t.append(H)
            if check_exclusions(t, exclusions) == False: forbidden_graph_k.append(t)

    forbidden_graph_k_cleaned = remove_redundant(forbidden_graph_k)  

    return forbidden_graph_k_cleaned


In [21]:
forbidden_graphs.insert(1,find_forbidden_1())
forbidden_graphs.insert(2,find_forbidden_k(2))
forbidden_graphs.insert(3,find_forbidden_k(3))

In [22]:
print(len(forbidden_graphs[3]))

30


In [235]:
f1 = open(r"1_case.txt","w")
for case in forbidden_graphs[1]:
    f1.write(case.graph6_string())
    f1.write("\n")
f1.close()

f2 = open(r"2_case.txt","w")
for case in forbidden_graphs[2]:
    f2.write(f"{case[0].graph6_string()},{case[1].graph6_string()}")
    f2.write("\n")
f2.close()

f3 = open(r"3_case.txt","w")
for case in forbidden_graphs[3]:
    f3.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()}")
    f3.write("\n")
f3.close()

f4 = open(r"4_case.txt","w")
for case in forbidden_graphs[4]:
    f4.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()},{case[3].graph6_string()}")
    f4.write("\n")
f4.close()

f5 = open(r"5_case.txt","w")
for case in forbidden_graphs[5]:
    f5.write(f"{case[0].graph6_string()},{case[1].graph6_string()},{case[2].graph6_string()},{case[3].graph6_string()},{case[4].graph6_string()}")
    f5.write("\n")
f5.close()